In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf

sess=tf.Session()

def unpickle(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
        return data

def load_cifar10_data(data_dir):
    train_data = None
    train_labels = []
    
    for i in range(1,6):
        data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
        if i==1:
            train_data = data_dic['data']
        else:
            train_data = np.vstack((train_data, data_dic['data']))
        train_labels += data_dic['labels']
    test_data_dic = unpickle(data_dir + "/test_batch")
    test_data = test_data_dic['data']
    test_labels = test_data_dic['labels']
    
    train_data = train_data.reshape((len(train_data), 3, 32, 32))
    train_data = np.rollaxis(train_data, 1, 4)
    train_labels = np.array(train_labels)
    
    test_data = test_data.reshape((len(test_data), 3, 32, 32))
    test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)
    
    return train_data, train_labels, test_data, test_labels

data_dir = 'Data/cifar-10-batches-py'
train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

print("Training data set : %3d, Test data Set : %3d" %(train_data.shape[0], test_data.shape[0]))

Training data set : 50000, Test data Set : 10000


# ONE HOT ENCODING

In [58]:
trl = tf.one_hot(train_labels, 10, 1, 0, -1)
train_label = sess.run(trl)
tsl = tf.one_hot(test_labels, 10, 1, 0, -1)
test_label = sess.run(tsl)

# PARAMETERS

In [69]:
n_input   = train_data.shape[1]*train_data.shape[2]
n_output  = 10
n_indepth = train_data.shape[3]

Ker1size  = 11
Ker1depth = 96
Ker2size  = 5
Ker2depth = 256
Ker3size  = 3
Ker3depth = 384
Ker4size  = 3
Ker4depth = 256

Map_res = 1024/(4*4*4*4)
FC1size   = 1024

learning_rate = 0.9

weights = {
    'wc1': tf.Variable(tf.random_normal([Ker1size, Ker1size, n_indepth, Ker1depth], mean=1,stddev=0.1)),
    'wc2': tf.Variable(tf.random_normal([Ker2size, Ker2size, Ker1depth, Ker2depth], stddev=0.1)),
    'wc3': tf.Variable(tf.random_normal([Ker3size, Ker3size, Ker2depth, Ker3depth], stddev=0.1)),
    'wc4': tf.Variable(tf.random_normal([Ker4size, Ker4size, Ker3depth, Ker4depth], stddev=0.1)),    
    
    'wd1': tf.Variable(tf.random_normal([(int)(Map_res*Ker4depth), FC1size], stddev=0.1)),
    'wd2': tf.Variable(tf.random_normal([FC1size, n_output], stddev=0.1))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([Ker1depth], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([Ker2depth], stddev=0.1)),
    'bc3': tf.Variable(tf.random_normal([Ker3depth], stddev=0.1)),
    'bc4': tf.Variable(tf.random_normal([Ker4depth], stddev=0.1)),
    
    'bd1': tf.Variable(tf.random_normal([FC1size], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}

# DEFINE NETWORK

In [70]:
def conv_sj(_input, _w, _b, _keepratio):
    #CONV LAYER 1
    conv1  = tf.nn.conv2d(_input, _w['wc1'], strides=[1,1,1,1], padding='SAME')
    mean,var = tf.nn.moments(conv1, [0,1,2])
    conv1n = tf.nn.batch_normalization(conv1, mean, var, 0, 1, 0.0001)
    conv1a = tf.nn.relu(tf.nn.bias_add(conv1n, _b['bc1']))
    pool1  = tf.nn.max_pool(conv1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    pool1d = tf.nn.dropout(pool1, _keepratio)
    conv1o = pool1d
    
    #CONV LAYER 2
    conv2  = tf.nn.conv2d(conv1o, _w['wc2'], strides=[1,1,1,1], padding='SAME')
    mean,var = tf.nn.moments(conv2, [0,1,2])
    conv2n = tf.nn.batch_normalization(conv2, mean, var, 0, 1, 0.0001)
    conv2a = tf.nn.relu(tf.nn.bias_add(conv2n, _b['bc2']))
    pool2  = tf.nn.max_pool(conv2a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    pool2d = tf.nn.dropout(pool2, _keepratio)
    conv2o = pool2d
    
    #CONV LAYER 3
    conv3  = tf.nn.conv2d(conv2o, _w['wc3'], strides=[1,1,1,1], padding='SAME')
    mean,var = tf.nn.moments(conv3, [0,1,2])
    conv3n = tf.nn.batch_normalization(conv3, mean, var, 0, 1, 0.0001)
    conv3a = tf.nn.relu(tf.nn.bias_add(conv3n, _b['bc3']))
    pool3  = tf.nn.max_pool(conv3a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    pool3d = tf.nn.dropout(pool3, _keepratio)
    conv3o = pool3d
    
    #CONV LAYER 4
    conv4  = tf.nn.conv2d(conv3o, _w['wc4'], strides=[1,1,1,1], padding='SAME')
    mean,var = tf.nn.moments(conv4, [0,1,2])
    conv4n = tf.nn.batch_normalization(conv4, mean, var, 0, 1, 0.0001)
    conv4a = tf.nn.relu(tf.nn.bias_add(conv4n, _b['bc4']))
    pool4  = tf.nn.max_pool(conv4a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    pool4d = tf.nn.dropout(pool4, _keepratio)
    conv4o = pool4d
    
    #VECTORIZE
    dense1 = tf.reshape(conv4o, [-1, _w['wd1'].get_shape().as_list()[0]])
    
    #FULLY CONNECTED LAYER 1
    fc1    = tf.nn.relu(tf.add(tf.matmul(dense1, _w['wd1']), _b['bd1']))
    fc1d   = tf.nn.dropout(fc1, _keepratio)
    fc1o   = fc1d
    
    #FULLY CONNECTED LAYER 2
    fc2    = tf.nn.relu(tf.add(tf.matmul(fc1o, _w['wd2']), _b['bd2']))
    out    = fc2
    
    #RETURNS
    _out = {
        'conv1': conv1, 'conv1n': conv1n, 'conv1a': conv1a, 'pool1': pool1, 'pool1d': pool1d, 'conv1o':conv1o,
        'conv2': conv2, 'conv2n': conv2n, 'conv2a': conv2a, 'pool2': pool2, 'pool2d': pool2d, 'conv2o':conv2o,       
        'conv3': conv3, 'conv3n': conv3n, 'conv3a': conv3a, 'pool3': pool3, 'pool3d': pool3d, 'conv3o':conv3o,
        'conv4': conv4, 'conv4n': conv4n, 'conv4a': conv4a, 'pool4': pool4, 'pool4d': pool4d, 'conv4o':conv4o,
        'dense1': dense1, 'fc1': fc1, 'fc1d': fc1d, 'fc1o': fc1o, 'fc2': fc2, 'out': out
    }
    return _out   
print(" Network Ready ")
                        

 Network Ready 


# DEFINE COMPUTATIONAL GRAPH

In [71]:
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

pred = conv_sj(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))
init = tf.initialize_all_variables()



Instructions for updating:
Use `tf.global_variables_initializer` instead.


# OPTIMIZE

In [72]:

sess.run(init)
print("initialized")

initialized


In [74]:
train_label.shape
sess.close

<bound method Session.close of <tensorflow.python.client.session.Session object at 0x7ff0def6efd0>>

In [67]:
training_epochs = 5
batch_size      = 100
display_step    = 1

print("Optimization started")
for epoch in range(training_epochs):
    avg_cost = 0.
    total_iter = int(train_data.shape[0]/batch_size)
    
    for i in range(total_iter):
        randidx = np.random.randint(50000, size=batch_size)
        batch_xs = train_data[randidx,:,:,:]
        batch_ys = train_label[randidx,:]
        
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio: 0.5})
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})/total_iter
        
        
    if epoch % display_step ==0 or epoch == training_epochs-1:
        print("epoch: %03d/%03d, cost: %.4f" %(epoch, training_epochs, avg_cost))
        training_acc = sess.run(accr, feed_dict={x: train_data[0:10000,:,:,:], y: train_label[0:10000,:], keepratio:1.})
        test_acc     = sess.run(accr, feed_dict={x: test_data[0:5000,:,:,:], y: test_label[0:5000,:], keepratio:1.})
        print("TR acc : %.3f, TS acc : %.3f" % (training_acc, test_acc))

Optimization started
epoch: 000/005, cost: 3.0455
TR acc : 0.100, TS acc : 0.098
epoch: 001/005, cost: 2.3026
TR acc : 0.101, TS acc : 0.098
epoch: 002/005, cost: 2.3026
TR acc : 0.100, TS acc : 0.098
epoch: 003/005, cost: 2.3026
TR acc : 0.100, TS acc : 0.098
epoch: 004/005, cost: 2.3026
TR acc : 0.100, TS acc : 0.098


In [68]:
a=weights['wc1'][:,:,1,1]
aout=sess.run(a)
aout

array([[ 1.43556511,  1.36108935,  1.34101105,  1.43881857,  1.12045872,
         1.17166495,  1.0675    ,  1.11042464,  1.13888204,  1.34621549,
         1.12239218],
       [ 1.36114812,  1.34451568,  1.30982232,  1.07686996,  1.1096766 ,
         1.44167006,  1.2124238 ,  1.07516694,  0.90495706,  0.98410738,
         1.48151314],
       [ 0.52257687,  0.50284803,  0.61970228,  0.50736535,  0.84917438,
         1.08278418,  0.95618892,  0.9587155 ,  0.91195107,  1.08326447,
         1.12626624],
       [ 0.69164801,  0.60172874,  0.57764882,  0.55144191,  0.88742298,
         1.09382904,  1.14436376,  1.25941956,  1.01933169,  0.92493349,
         1.09037006],
       [ 1.05796576,  1.03275025,  0.70027667,  0.78522104,  0.60386992,
         1.06440282,  1.16064501,  0.81264949,  1.04575765,  1.20180297,
         1.05160117],
       [ 1.23734665,  1.389817  ,  0.87278873,  0.83719671,  0.51160049,
         1.43569088,  1.30859065,  1.09812975,  1.27055585,  1.12793684,
         1.399